In [2]:
%store -r X_small y indexes X

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import numpy as np
import pandas as pd
import glob
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import copy
from sklearn.ensemble import RandomForestClassifier

In [3]:
def load_data():
    path = f'./python'
    files = glob.glob(path + '/test*.npy')
    files = sorted(files)
    
    result = []
    labels = [file.split('/')[-1].replace('.npy', '') for file in files]
    
    for i, file in enumerate(files):
        print(f"Loaded {i + 1}/{len(files)} test files.", end="\r")
        data = np.load(file)
        result.append(data)
        
    result = np.array(result)
    
    return result, labels

In [4]:
test_data, labels = load_data()

In [5]:
sample = test_data[0]
labels

['test00',
 'test01',
 'test02',
 'test03',
 'test04',
 'test05',
 'test06',
 'test07',
 'test08',
 'test09',
 'test10',
 'test11',
 'test12',
 'test13',
 'test14',
 'test15']

# Neural Network

In [13]:
# class ComplexNet(nn.Module):
#     def __init__(self):
#         super(ComplexNet, self).__init__()
        
#         # CNN layers
#         self.conv1 = nn.Conv1d(1, 256, kernel_size=3, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm1d(256)
#         self.conv2 = nn.Conv1d(256, 128, kernel_size=3, stride=1, padding=1)
#         self.bn2 = nn.BatchNorm1d(128)
        
#         # RNN layer
#         self.gru = nn.GRU(input_size=128, hidden_size=64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
        
#         # Fully connected layers
#         self.fc1 = nn.Linear(64*2, 128)  # 2 for bidirectional outputs
#         self.dropout = nn.Dropout(0.2)
#         self.fc2 = nn.Linear(128, 7)
    
#     def forward(self, x):
#         # CNN layers
#         x = x.unsqueeze(1)  # (N, 548) -> (N, 1, 548)
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
        
#         # Global Average Pooling
#         x = torch.mean(x, 2)
        
#         # RNN layer
#         x = x.unsqueeze(1)  # (N, 128) -> (N, 1, 128) for RNN
#         _, h_n = self.gru(x)
#         x = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)  # Concat the last hidden state of layers
        
#         # Fully connected layers
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
        
#         return x

In [17]:
class BirdClassifier(nn.Module):
    def __init__(self):
        super(BirdClassifier, self).__init__()
        
        self.fc1 = nn.Linear(548, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)

        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)

        self.fc5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(64, 7)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.bn3(self.fc3(x)))
        x = F.relu(self.bn4(self.fc4(x)))
        x = F.relu(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        x = self.out(x)
        return x


In [20]:
# Define the loss function and optimizer
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Define the hyperparameters
learning_rate = 1e-5
batch_size = 128
num_epochs = 200

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).long().to(device)
y_test = torch.from_numpy(y_test).long().to(device)

# Create the DataLoader for training and test sets
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = BirdClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Initialize variables for saving the best model
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

# Train the model
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.to(torch.float32) / len(train_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}")

    # Evaluate the model on the test set
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects.to(torch.float32) / len(test_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {epoch_loss:.4f}, Test Acc: {epoch_acc:.4f}")

    # Save the model if it has a higher accuracy than our current best
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

# Load the best model weights
model.load_state_dict(best_model_wts)

Epoch 1/200, Train Loss: 1.6755, Train Acc: 0.6168
Epoch 1/200, Test Loss: 1.9590, Test Acc: 0.0464
Epoch 2/200, Train Loss: 1.3975, Train Acc: 0.7505
Epoch 2/200, Test Loss: 3.7220, Test Acc: 0.0464
Epoch 3/200, Train Loss: 1.2229, Train Acc: 0.7765
Epoch 3/200, Test Loss: 2.0160, Test Acc: 0.0464
Epoch 4/200, Train Loss: 1.0928, Train Acc: 0.7864
Epoch 4/200, Test Loss: 1.8395, Test Acc: 0.0464
Epoch 5/200, Train Loss: 1.0044, Train Acc: 0.7943
Epoch 5/200, Test Loss: 1.6642, Test Acc: 0.7805
Epoch 6/200, Train Loss: 0.9272, Train Acc: 0.7992
Epoch 6/200, Test Loss: 1.5211, Test Acc: 0.7805
Epoch 7/200, Train Loss: 0.8690, Train Acc: 0.8013
Epoch 7/200, Test Loss: 5.9780, Test Acc: 0.7805
Epoch 8/200, Train Loss: 0.8171, Train Acc: 0.8053
Epoch 8/200, Test Loss: 1.5490, Test Acc: 0.7805
Epoch 9/200, Train Loss: 0.7783, Train Acc: 0.8068
Epoch 9/200, Test Loss: 1.3178, Test Acc: 0.7805
Epoch 10/200, Train Loss: 0.7352, Train Acc: 0.8099
Epoch 10/200, Test Loss: 1.2373, Test Acc: 0.780

KeyboardInterrupt: 

In [15]:
sub = ""

for i, sample in enumerate(test_data):
    y_test = sample[:, indexes]
    y_pred = clf.predict(y_test)
    y_pred = y_pred.astype(str)
    
    y_str = ','.join(y_pred)
    result = f'{labels[i]},{y_str}\n'
    sub += result
    
sub

'test00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,0,0,1,0,1,1,0,0,0,1,1,1,3,0,0,0,0,1,0,1,1,0,0,0,1,0,1,1,0,0,0,0,5,0,0,5,0,0,0,0,0,0,0,0,0,0,5,0,0,5,0,0,5,0,0,0,5,0,0,5,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,0,1,1,1,1,0,0,0,0,1,0,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,3,

In [16]:
with open ('./submission_random_forest.csv', 'w') as file:
    file.write(sub)

# Neural Networks

# Simple Neural Network

In [26]:
# Set the device (GPU if available, else CPU)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

# Define the neural network architecture
class BirdClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BirdClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

class BirdClassifier(nn.Module):
    def __init__(self):
        super(BirdClassifier, self).__init__()
        
        self.fc1 = nn.Linear(50, 256)
        self.bn1 = nn.BatchNorm1d(256)
        
        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)

        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)

        self.fc5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(64, 7)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.bn3(self.fc3(x)))
        x = F.relu(self.bn4(self.fc4(x)))
        x = F.relu(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        x = self.out(x)
        return x

# Define the hyperparameters
input_size = 50
hidden_size = 64
num_classes = 7
learning_rate = 1e-5
batch_size = 128
num_epochs = 200

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_small, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).long().to(device)
y_test = torch.from_numpy(y_test).long().to(device)

# Create the DataLoader for training and test sets
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create an instance of the BirdClassifier model
model = BirdClassifier().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
losses = []
accuracies = []

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    with torch.no_grad():
        y_pred = []
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.cpu().numpy().tolist())
    
    acc = accuracy_score(y_test.cpu().numpy(), y_pred)
    losses.append(loss.item())
    accuracies.append(acc)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")

mps
Epoch [1/200], Loss: 1.5031, Accuracy: 0.1818
Epoch [2/200], Loss: 1.3470, Accuracy: 0.7680
Epoch [3/200], Loss: 1.4352, Accuracy: 0.7870
Epoch [4/200], Loss: 1.2017, Accuracy: 0.8215
Epoch [5/200], Loss: 0.9354, Accuracy: 0.8329
Epoch [6/200], Loss: 1.0176, Accuracy: 0.8315
Epoch [7/200], Loss: 0.7755, Accuracy: 0.8435
Epoch [8/200], Loss: 0.7470, Accuracy: 0.8433
Epoch [9/200], Loss: 0.6076, Accuracy: 0.8470
Epoch [10/200], Loss: 0.6145, Accuracy: 0.8495
Epoch [11/200], Loss: 0.5741, Accuracy: 0.8522
Epoch [12/200], Loss: 0.5967, Accuracy: 0.8522
Epoch [13/200], Loss: 0.5800, Accuracy: 0.8587
Epoch [14/200], Loss: 0.6586, Accuracy: 0.8637
Epoch [15/200], Loss: 0.6737, Accuracy: 0.8724
Epoch [16/200], Loss: 0.4131, Accuracy: 0.8763
Epoch [17/200], Loss: 0.4084, Accuracy: 0.8811
Epoch [18/200], Loss: 0.4382, Accuracy: 0.8798
Epoch [19/200], Loss: 0.4282, Accuracy: 0.8806
Epoch [20/200], Loss: 0.3854, Accuracy: 0.8851
Epoch [21/200], Loss: 0.4006, Accuracy: 0.8812
Epoch [22/200], Lo

Epoch [174/200], Loss: 0.1129, Accuracy: 0.9295
Epoch [175/200], Loss: 0.3932, Accuracy: 0.9270
Epoch [176/200], Loss: 0.4085, Accuracy: 0.9381
Epoch [177/200], Loss: 0.2824, Accuracy: 0.9384
Epoch [178/200], Loss: 0.2927, Accuracy: 0.9422
Epoch [179/200], Loss: 0.4951, Accuracy: 0.9372
Epoch [180/200], Loss: 0.1896, Accuracy: 0.9351
Epoch [181/200], Loss: 0.2367, Accuracy: 0.9453
Epoch [182/200], Loss: 0.1154, Accuracy: 0.9396
Epoch [183/200], Loss: 0.1974, Accuracy: 0.9425
Epoch [184/200], Loss: 0.0991, Accuracy: 0.9448
Epoch [185/200], Loss: 0.1095, Accuracy: 0.9442
Epoch [186/200], Loss: 0.2934, Accuracy: 0.9438
Epoch [187/200], Loss: 0.3028, Accuracy: 0.9389
Epoch [188/200], Loss: 0.1035, Accuracy: 0.9372
Epoch [189/200], Loss: 0.1634, Accuracy: 0.9422
Epoch [190/200], Loss: 0.2681, Accuracy: 0.9361
Epoch [191/200], Loss: 0.2547, Accuracy: 0.9400
Epoch [192/200], Loss: 0.2086, Accuracy: 0.9395
Epoch [193/200], Loss: 0.3105, Accuracy: 0.9419
Epoch [194/200], Loss: 0.1298, Accuracy:

In [112]:
class AudioNet(nn.Module):
    def __init__(self):
        super(AudioNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128*548, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 7)

    def forward(self, x):
        x = x.view(x.shape[0], 1, -1)  # Reshape the input data
        x = F.relu(self.conv1(x))
        x = x.view(x.shape[0], -1)  # Flatten the data
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x


In [119]:
class AudioNet(nn.Module):
    def __init__(self):
        super(AudioNet, self).__init__()

        self.lstm = nn.LSTM(input_size=50, hidden_size=256, num_layers=2, batch_first=True, dropout=0.5)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 7)

    def forward(self, x):
        # LSTM layer expects input of shape (batch_size, seq_length, input_size)
        x = x.view(x.shape[0], 1, -1)  
        
        output, _ = self.lstm(x)
        
        # Use the output from the last time step
        output = output[:, -1, :]
        
        output = F.relu(self.fc1(output))
        output = F.log_softmax(self.fc2(output), dim=1)

        return output



In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

# Define the hyperparameters
learning_rate = 1e-5
batch_size = 128
num_epochs = 200

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_small, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).long().to(device)
y_test = torch.from_numpy(y_test).long().to(device)

# Create the DataLoader for training and test sets
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create an instance of the BirdClassifier model
model = AudioNet().to(device)

# Define the loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
losses = []
accuracies = []

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    with torch.no_grad():
        y_pred = []
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.cpu().numpy().tolist())
    
    acc = accuracy_score(y_test.cpu().numpy(), y_pred)
    losses.append(loss.item())
    accuracies.append(acc)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")

# RNN

In [137]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

# Define the hyperparameters
learning_rate = 1e-5
batch_size = 32
num_epochs = 200

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_small, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_train = torch.from_numpy(y_train).long().to(device)
y_test = torch.from_numpy(y_test).long().to(device)

# Create the DataLoader for training and test sets
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


mps


In [138]:
class AudioNetRNN(nn.Module):
    def __init__(self):
        super(AudioNetRNN, self).__init__()

        self.lstm = nn.LSTM(input_size=50, hidden_size=256, num_layers=2, batch_first=True, dropout=0.5)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 7)

    def forward(self, x):
        # LSTM layer expects input of shape (batch_size, seq_length, input_size)
        x = x.view(x.shape[0], 1, -1)  
        
        output, _ = self.lstm(x)
        
        # Use the output from the last time step
        output = output[:, -1, :]
        
        output = F.relu(self.fc1(output))
        output = F.log_softmax(self.fc2(output), dim=1)

        return output

In [139]:
# Define the loss function and optimizer
model = AudioNetRNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Initialize variables for saving the best model
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

# Train the model
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.to(torch.float32) / len(train_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}")

    # Evaluate the model on the test set
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects.to(torch.float32) / len(test_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {epoch_loss:.4f}, Test Acc: {epoch_acc:.4f}")

    # Save the model if it has a higher accuracy than our current best
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

# Load the best model weights
model.load_state_dict(best_model_wts)

Epoch 1/200, Train Loss: 1.0348, Train Acc: 0.6979
Epoch 1/200, Test Loss: 0.7387, Test Acc: 0.7804
Epoch 2/200, Train Loss: 0.6973, Train Acc: 0.7845
Epoch 2/200, Test Loss: 0.6583, Test Acc: 0.7811
Epoch 3/200, Train Loss: 0.6106, Train Acc: 0.7922
Epoch 3/200, Test Loss: 0.5695, Test Acc: 0.8016
Epoch 4/200, Train Loss: 0.5404, Train Acc: 0.8164
Epoch 4/200, Test Loss: 0.5171, Test Acc: 0.8234
Epoch 5/200, Train Loss: 0.5048, Train Acc: 0.8304
Epoch 5/200, Test Loss: 0.4882, Test Acc: 0.8343
Epoch 6/200, Train Loss: 0.4812, Train Acc: 0.8374
Epoch 6/200, Test Loss: 0.4744, Test Acc: 0.8382
Epoch 7/200, Train Loss: 0.4678, Train Acc: 0.8415
Epoch 7/200, Test Loss: 0.4630, Test Acc: 0.8391
Epoch 8/200, Train Loss: 0.4533, Train Acc: 0.8453
Epoch 8/200, Test Loss: 0.4461, Test Acc: 0.8453
Epoch 9/200, Train Loss: 0.4429, Train Acc: 0.8477
Epoch 9/200, Test Loss: 0.4385, Test Acc: 0.8467
Epoch 10/200, Train Loss: 0.4307, Train Acc: 0.8509
Epoch 10/200, Test Loss: 0.4250, Test Acc: 0.851

Epoch 81/200, Test Loss: 0.2035, Test Acc: 0.9306
Epoch 82/200, Train Loss: 0.1968, Train Acc: 0.9336
Epoch 82/200, Test Loss: 0.2008, Test Acc: 0.9319
Epoch 83/200, Train Loss: 0.1986, Train Acc: 0.9327
Epoch 83/200, Test Loss: 0.1976, Test Acc: 0.9329
Epoch 84/200, Train Loss: 0.1968, Train Acc: 0.9333
Epoch 84/200, Test Loss: 0.1942, Test Acc: 0.9344
Epoch 85/200, Train Loss: 0.1953, Train Acc: 0.9346
Epoch 85/200, Test Loss: 0.1937, Test Acc: 0.9343
Epoch 86/200, Train Loss: 0.1946, Train Acc: 0.9337
Epoch 86/200, Test Loss: 0.2047, Test Acc: 0.9309
Epoch 87/200, Train Loss: 0.1961, Train Acc: 0.9338
Epoch 87/200, Test Loss: 0.1934, Test Acc: 0.9347
Epoch 88/200, Train Loss: 0.1925, Train Acc: 0.9356
Epoch 88/200, Test Loss: 0.1900, Test Acc: 0.9364
Epoch 89/200, Train Loss: 0.1932, Train Acc: 0.9339
Epoch 89/200, Test Loss: 0.1953, Test Acc: 0.9340
Epoch 90/200, Train Loss: 0.1943, Train Acc: 0.9346
Epoch 90/200, Test Loss: 0.1948, Test Acc: 0.9340
Epoch 91/200, Train Loss: 0.1930

Epoch 161/200, Train Loss: 0.1650, Train Acc: 0.9435
Epoch 161/200, Test Loss: 0.1715, Test Acc: 0.9436
Epoch 162/200, Train Loss: 0.1629, Train Acc: 0.9442
Epoch 162/200, Test Loss: 0.1702, Test Acc: 0.9431
Epoch 163/200, Train Loss: 0.1612, Train Acc: 0.9449
Epoch 163/200, Test Loss: 0.1694, Test Acc: 0.9444
Epoch 164/200, Train Loss: 0.1618, Train Acc: 0.9443
Epoch 164/200, Test Loss: 0.1696, Test Acc: 0.9439
Epoch 165/200, Train Loss: 0.1597, Train Acc: 0.9455
Epoch 165/200, Test Loss: 0.1710, Test Acc: 0.9423
Epoch 166/200, Train Loss: 0.1615, Train Acc: 0.9452
Epoch 166/200, Test Loss: 0.1752, Test Acc: 0.9406
Epoch 167/200, Train Loss: 0.1616, Train Acc: 0.9458
Epoch 167/200, Test Loss: 0.1729, Test Acc: 0.9432
Epoch 168/200, Train Loss: 0.1605, Train Acc: 0.9453
Epoch 168/200, Test Loss: 0.1688, Test Acc: 0.9440
Epoch 169/200, Train Loss: 0.1604, Train Acc: 0.9460
Epoch 169/200, Test Loss: 0.1687, Test Acc: 0.9432
Epoch 170/200, Train Loss: 0.1596, Train Acc: 0.9452
Epoch 170/2

<All keys matched successfully>

In [142]:
model.eval()

for i, sample in enumerate(test_data):
    y_test = sample[:, indexes].to(device)
    y_pred = model(y_test)
    print(y_pred)
#     y_pred = y_pred.astype(str)
    
#     y_str = ','.join(y_pred)
#     result = f'{labels[i]},{y_str}\n'
#     sub += result

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [149]:
def predict(model, X_unlabeled):
    model.eval()  # Ensure the model is in evaluation mode
    predictions = []

    with torch.no_grad():
        for inputs in X_unlabeled:
            inputs = inputs.to(device)
            outputs = model(inputs.unsqueeze(0))  # Pass data through model, ensure the input tensor is (batch_size, features) shape
            _, predicted = torch.max(outputs, 1)  # Get the predicted classes
            predictions.extend(predicted.cpu().numpy().tolist())  # Add the predictions to our list

    return predictions

# Assuming X_unlabeled is a numpy array
X_unlabeled = torch.from_numpy(test_data[:, :, indexes]).float().to(device)

predictions = predict(model, X_unlabeled)


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [148]:
test_data[:, :, indexes].shape

(16, 3000, 50)

In [169]:
# Assuming X_unlabeled is a numpy array
X_unlabeled = torch.from_numpy(test_data[:, :, indexes]).float().to(device)

# Create a DataLoader for the unlabeled data
unlabeled_dataset = TensorDataset(X_unlabeled)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)


# Prediction function
def predict(model, unlabeled_loader):
    model.eval()  # Ensure the model is in evaluation mode
    result = []
    predictions = []

    with torch.no_grad():
        for sample in test_data[:, :, indexes]:
            X_unlabeled = torch.from_numpy(sample).float().to(device)
            # Create a DataLoader for the unlabeled data
            unlabeled_dataset = TensorDataset(X_unlabeled)
            unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)
            
            for inputs in unlabeled_loader:
                inputs = inputs[0].to(device)
                outputs = model(inputs)  # Pass data through model
                _, predicted = torch.max(outputs, 1)  # Get the predicted classes
                predictions.extend(predicted.cpu().numpy().tolist())  # Add the predictions to our list
            
            result.append(predictions)
            predictions = []

    return np.array(result)

# Predict
predictions = predict(model, unlabeled_loader)


In [179]:
sub = ""

for i, prediction in enumerate(predictions):
    y_pred = prediction.astype(str)
    
    y_str = ','.join(y_pred)
    result = f'{labels[i]},{y_str}\n'
    sub += result

In [180]:
sub

'test00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,0,0,1,0,1,3,0,0,0,1,1,1,3,0,0,0,0,3,0,3,1,0,0,0,1,0,3,1,0,0,0,0,5,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,1,0,0,0,1,1,3,3,0,0,0,0,1,0,3,3,2,0,0,1,1,1,3,0,0,0,2,1,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,0,0,0,0,0,0,1,1,

In [181]:
# with open ('./submission_neural_network.csv', 'w') as file:
#     file.write(sub)

In [3]:
indexes

array([537,  22, 236,  25, 286, 538, 265,  29, 288,  21,  20,  23,  61,
       290,  84, 271, 151,  83, 402, 274, 536, 152, 289, 145, 139, 146,
       137, 301, 264, 539,  18, 293, 306,  31, 270,  19, 138, 272,  16,
       291,  17, 237, 176, 287, 294,  27, 292,  24,  46,  80])